In [1]:
import os
# import tensorflow.compat.v2 as tf
import tensorflow as tf
import json
from tensorflow.keras import datasets, layers, models, metrics
from minio import Minio
from minio.error import ResponseError

In [2]:
!wget https://dl.min.io/client/mc/release/linux-amd64/mc
!chmod +x mc
!sudo mv mc /usr/bin
!mc config host add myminio http://minio-service.kubeflow:9000 minio minio123
!mc config host list

--2020-12-03 16:06:47--  https://dl.min.io/client/mc/release/linux-amd64/mc
Resolving dl.min.io (dl.min.io)... 178.128.69.202
Connecting to dl.min.io (dl.min.io)|178.128.69.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19623936 (19M) [application/octet-stream]
Saving to: ‘mc’

mc                  100%[===================>]  18.71M  8.65MB/s    in 2.2s    

2020-12-03 16:06:50 (8.65 MB/s) - ‘mc’ saved [19623936/19623936]

Added `myminio` successfully.
gcs    
  URL       : https://storage.googleapis.com
  AccessKey : YOUR-ACCESS-KEY-HERE
  SecretKey : YOUR-SECRET-KEY-HERE
  API       : S3v2
  Path      : dns

local  
  URL       : http://localhost:9000
  AccessKey : 
  SecretKey : 
  API       : 
  Path      : auto

myminio
  URL       : http://minio-service.kubeflow:9000
  AccessKey : minio
  SecretKey : minio123
  API       : s3v4
  Path      : auto

play   
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswR

In [3]:
!mc rm --force --recursive myminio/model/
!mc rm myminio/model/mnist-model/assets/
!mc rm myminio/model/mnist-model/
!mc rb myminio/model

!mc rm --force --recursive myminio/dataset
!mc rm myminio/dataset/mnist/
!mc rb myminio/dataset

!mc tree myminio

!rm -f /tmp/mnist.npz

mc: <ERROR> Failed to remove `myminio/model/` recursively. The specified bucket does not exist
mc: <ERROR> Failed to remove `myminio/model/mnist-model/assets/`. The specified bucket does not exist
mc: <ERROR> Failed to remove `myminio/model/mnist-model/`. The specified bucket does not exist
mc: <ERROR> Unable to validate target `myminio/model`. Bucket `model` does not exist.
mc: <ERROR> Failed to remove `myminio/dataset` recursively. The specified bucket does not exist
mc: <ERROR> Failed to remove `myminio/dataset/mnist/`. The specified bucket does not exist
mc: <ERROR> Unable to validate target `myminio/dataset`. Bucket `dataset` does not exist.
myminio
└─ mlpipeline


In [4]:
!mc mb myminio/dataset
!mc mb myminio/model
!mc tree myminio

Bucket created successfully `myminio/dataset`.
Bucket created successfully `myminio/model`.
myminio
├─ dataset
├─ mlpipeline
└─ model


In [5]:
!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
!mc cp mnist.npz myminio/dataset/mnist/

--2020-12-03 16:07:02--  https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 64.233.189.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz.13’

mnist.npz.13        100%[===================>]  10.96M  --.-KB/s    in 0.06s   

2020-12-03 16:07:02 (173 MB/s) - ‘mnist.npz.13’ saved [11490434/11490434]

mnist.npz:     10.96 MiB / 10.96 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 123.74 MiB/s 0s

In [6]:
minioClient = Minio('minio-service.kubeflow:9000',
              access_key='minio', secret_key='minio123', secure=False)
try:
    data = minioClient.get_object('dataset', 'mnist/mnist.npz')
    with open('/tmp/mnist.npz', 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='/tmp/mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
dropout_rate = 0.3
learning_rate = 0.1

In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

sgd = tf.keras.optimizers.SGD(lr=learning_rate,
                                  decay=1e-6,
                                  momentum=0.9,
                                  nesterov=True)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3)
score = model.evaluate(x_test,  y_test, verbose=2)
loss = score[0]
accuracy = score[1]

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 5s 79us/sample - loss: 0.4372 - accuracy: 0.8711
Epoch 2/3
60000/60000 [==============================] - 4s 74us/sample - loss: 0.3522 - accuracy: 0.9040
Epoch 3/3
60000/60000 [==============================] - 4s 74us/sample - loss: 0.3206 - accuracy: 0.9163
10000/10000 - 1s - loss: 0.1922 - accuracy: 0.9470


In [9]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})    
model.save("s3://model/mnist-model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: s3://model/mnist-model/assets


In [10]:
print(loss)
print(accuracy)

0.19220291791511698
0.947
